In [4]:
%%sql
CREATE TABLE IF NOT EXISTS gold_dim_user (
  client_id            BIGINT       NOT NULL,

  current_age          INT,
  retirement_age       INT,
  birth_year           INT,
  birth_month          INT,
  gender               STRING,
  address              STRING,
  latitude             DECIMAL(9,6),
  longitude            DECIMAL(9,6),
  per_capita_income    DECIMAL(18,2),
  yearly_income        DECIMAL(18,2),
  total_debt           DECIMAL(18,2),
  credit_score         INT,
  num_credit_cards     INT,

  -- Audit / lineage (Silver)
  source_file          STRING       NOT NULL,
  ingestion_date       DATE         NOT NULL,
  ingestion_ts         TIMESTAMP    NOT NULL,
  record_hash          STRING       NOT NULL,

  -- Audit (Gold)
  gold_run_id          STRING       NOT NULL,
  gold_load_ts         TIMESTAMP    NOT NULL
)
USING DELTA;


StatementMeta(, b6785450-1c51-48dd-856e-68d50242a1bb, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [5]:
%%sql
CREATE TABLE IF NOT EXISTS gold_dim_card (
  card_id               BIGINT        NOT NULL,

  client_id             BIGINT,
  card_brand            STRING,
  card_type             STRING,
  card_number           STRING,
  expires_raw           STRING,
  expires_month         DATE,
  cvv                   STRING,
  has_chip              BOOLEAN,
  num_cards_issued      INT,
  credit_limit          DECIMAL(18,2),
  acct_open_date        DATE,
  year_pin_last_changed INT,
  card_on_dark_web      BOOLEAN,

  -- Audit / lineage (Silver)
  source_file           STRING       NOT NULL,
  ingestion_date        DATE         NOT NULL,
  ingestion_ts          TIMESTAMP    NOT NULL,
  record_hash           STRING       NOT NULL,

  -- Audit (Gold)
  gold_run_id           STRING       NOT NULL,
  gold_load_ts          TIMESTAMP    NOT NULL
)
USING DELTA;


StatementMeta(, b6785450-1c51-48dd-856e-68d50242a1bb, 6, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [6]:
%%sql
CREATE TABLE IF NOT EXISTS gold_dim_mcc (
  mcc_code          STRING      NOT NULL,
  mcc_description   STRING,

  -- Audit / lineage (Silver)
  source_file       STRING      NOT NULL,
  ingestion_date    DATE        NOT NULL,
  ingestion_ts      TIMESTAMP   NOT NULL,
  record_hash       STRING      NOT NULL,

  -- Audit (Gold)
  gold_run_id       STRING      NOT NULL,
  gold_load_ts      TIMESTAMP   NOT NULL
)
USING DELTA;


StatementMeta(, b6785450-1c51-48dd-856e-68d50242a1bb, 7, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
%%sql
CREATE TABLE IF NOT EXISTS gold_dim_date (
  date_id              INT         NOT NULL,   -- YYYYMMDD
  date_value           DATE        NOT NULL,

  day_of_month          INT,
  day_of_week_iso       INT,                  -- 1=Mon ... 7=Sun
  day_name              STRING,
  week_of_year          INT,
  month_number          INT,
  month_name            STRING,
  quarter_number        INT,
  year_number           INT,
  is_weekend            BOOLEAN,

  gold_run_id           STRING     NOT NULL,
  gold_load_ts          TIMESTAMP  NOT NULL
)
USING DELTA;


StatementMeta(, b6785450-1c51-48dd-856e-68d50242a1bb, 8, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [8]:
%%sql
CREATE TABLE IF NOT EXISTS gold_fact_transactions (
  transaction_id    BIGINT        NOT NULL,

  txn_ts            TIMESTAMP     NOT NULL,
  txn_date          DATE          NOT NULL,
  txn_month         DATE          NOT NULL,

  client_id         BIGINT        NOT NULL,
  card_id           BIGINT        NOT NULL,
  merchant_id       BIGINT        NOT NULL,
  mcc_code          STRING        NOT NULL,

  amount            DECIMAL(18,2) NOT NULL,
  use_chip          STRING,
  merchant_city     STRING,
  merchant_state    STRING,
  zip               STRING,
  error_code        INT           NOT NULL,
  is_success        BOOLEAN       NOT NULL,

  -- Audit / lineage (Silver)
  source_file       STRING,
  ingestion_date    DATE          NOT NULL,
  ingestion_ts      TIMESTAMP     NOT NULL,
  record_hash       STRING        NOT NULL,

  -- Audit (Gold)
  gold_run_id       STRING        NOT NULL,
  gold_load_ts      TIMESTAMP     NOT NULL
)
USING DELTA
PARTITIONED BY (txn_month);


StatementMeta(, b6785450-1c51-48dd-856e-68d50242a1bb, 9, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [1]:
%%sql
CREATE TABLE IF NOT EXISTS gold_anom_dim_card_orphans (
  -- same shape as dim_card (business+tech)
  card_id               BIGINT,
  client_id             BIGINT,
  card_brand            STRING,
  card_type             STRING,
  card_number           STRING,
  expires_raw           STRING,
  expires_month         DATE,
  cvv                   STRING,
  has_chip              BOOLEAN,
  num_cards_issued      INT,
  credit_limit          DECIMAL(18,2),
  acct_open_date        DATE,
  year_pin_last_changed INT,
  card_on_dark_web      BOOLEAN,

  source_file           STRING,
  ingestion_date        DATE,
  ingestion_ts          TIMESTAMP,
  record_hash           STRING,

  -- anomaly metadata
  anom_type             STRING,
  gold_run_id           STRING,
  gold_load_ts          TIMESTAMP
)
USING DELTA;


StatementMeta(, eb677dd8-3890-4fe4-83ec-f25d61c37dd1, 2, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [1]:
%%sql
CREATE TABLE IF NOT EXISTS gold_anomaly_event (
  -- identity
  anomaly_id        STRING      NOT NULL,   -- uuid-like string
  anom_domain       STRING      NOT NULL,   -- e.g. GOLD_FACT_TRANSACTIONS
  anom_type         STRING      NOT NULL,   -- e.g. ORPHAN_DIM_USER, ORPHAN_DIM_CARD, ...
  severity          STRING      NOT NULL,   -- INFO | WARN | ERROR | CRITICAL
  rule_id           STRING      NOT NULL,   -- stable rule identifier

  -- business keys / grain context (generic)
  transaction_id    BIGINT,                 -- populated for transaction anomalies
  client_id         BIGINT,
  card_id           BIGINT,
  mcc_code          STRING,
  txn_ts            TIMESTAMP,
  txn_date          DATE,
  txn_month         DATE,

  -- values / details
  metric_name       STRING,                 -- optional (e.g. "orphan_count")
  metric_value      DOUBLE,                 -- optional
  details_json      STRING,                 -- small JSON for debugging (keep compact)

  -- lineage (from Silver)
  source_file       STRING,
  ingestion_date    DATE,
  ingestion_ts      TIMESTAMP,

  -- gold audit
  gold_run_id       STRING      NOT NULL,
  gold_load_ts      TIMESTAMP   NOT NULL
)
USING DELTA;


StatementMeta(, fa7408da-a698-47b7-a942-ac90d1d5ff70, 2, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [2]:
%%sql
CREATE TABLE IF NOT EXISTS gold_anomaly_kpi (
  anom_domain     STRING      NOT NULL,
  anom_type       STRING      NOT NULL,
  severity        STRING      NOT NULL,
  rule_id         STRING      NOT NULL,
  anomaly_count   BIGINT      NOT NULL,
  gold_run_id     STRING      NOT NULL,
  gold_load_ts    TIMESTAMP   NOT NULL
)
USING DELTA;


StatementMeta(, fa7408da-a698-47b7-a942-ac90d1d5ff70, 3, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>